In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import InputLayer, Input, Dense
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.utils import plot_model\
# from tensorflow.keras.regularizers import Regularizer
import pandas as pd
import numpy as np

In [2]:
print(tf.__version__)

2.8.0


In [3]:
# !pip install git+https://github.com/tensorflow/docs

import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

In [4]:
from IPython import display
from matplotlib import pyplot as plt

import pathlib
import shutil
import tempfile

In [5]:
logdir = pathlib.Path(tempfile.mkdtemp())/'tensorboard_logs'

In [6]:
shutil.rmtree(logdir, ignore_errors=True)

In [7]:
gz = tf.keras.utils.get_file('HIGGS.csv.gz', 'http://mlphysics.ics.uci.edu/data/higgs/HIGGS.csv.gz')

2816417792/2816407858 [==============================] - 1644s 1us/step


In [8]:
FEATURES = 28